## RFM - анализ

Проведите RFM анализ. В каждом подсегменте поделите пользователей на 4 класса. Отсчитывайте количество дней, прошедших с момента последней покупки, с максимальной даты покупки в датасете.

Какое максимальное кол-во покупок было совершено одним пользователем?  
Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)    
Сколько пользователей попало в сегмент 111?  
Сколько пользователей попало в сегмент 311?  
В каком RFM-сегменте самое большое кол-во пользователей?  
В каком RFM-сегменте самое маленькое кол-во пользователей?  
Какое количество пользователей попало в самый малочисленный сегмент?  

In [1]:
# Импортируем бибилиотеки

import pandas as pd

In [2]:
# Считываем данные

df = pd.read_csv('RFM_ht_data.csv')
df.head()

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [3]:
# Проверяем  тип данных и меняем его на необходимый

df.dtypes

InvoiceNo        object
CustomerCode     object
InvoiceDate      object
Amount          float64
dtype: object

In [4]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerCode'] = df.CustomerCode.astype('str')
df['InvoiceNo'] = df.InvoiceNo.astype('str')

In [5]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

### Какое максимальное кол-во покупок было совершено одним пользователем?

In [6]:
df.CustomerCode.value_counts().max()

204

### Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

In [7]:
# Найдем последнюю дату в датасете

last_date = df.InvoiceDate.max()
last_date

Timestamp('2020-09-30 00:00:00')

In [8]:
# Посчитаем показатели recency, frequency и monetary для каждого пользователя

rfm = df.groupby('CustomerCode', as_index = False) \
        .agg({'InvoiceDate' : lambda x :(last_date - x.max()).days,
              'InvoiceNo' : lambda x: len(x),
              'Amount' : lambda x: x.sum()}) \
        .rename(columns = {'InvoiceDate' : 'recency',
                           'InvoiceNo' : 'frequency',
                           'Amount' : 'monetary'})
rfm.head()

,CustomerCode,recency,frequency,monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88


In [9]:
# Посмотрим распределение значений по квартилям

quantiles = rfm[['recency', 'frequency', 'monetary']].quantile(q = [0.25, 0.5, 0.75])
quantiles

,recency,frequency,monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


Верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М = 765

### Сколько пользователей попало в сегмент 111?

In [10]:
# Напишем функции для присвоения ранга:
# Чем меньше recency (чем недавнее совершена покупка), тем больше баллов начисляем.
# Чем меньше frequency и monetary (чем меньше кол-во и сумма покупок), тем меньше баллов начисляем.

def r_score(x):
    if x <= quantiles['recency'][0.25]:
        return 4
    elif x <= quantiles['recency'][0.5]:
        return 3
    elif x <= quantiles['recency'][0.75]:
        return 2
    else:
        return 1

In [11]:
def fm_score(x, c):
    if x <= quantiles[c][0.25]:
        return 1
    elif x <= quantiles[c][0.5]:
        return 2
    elif x <= quantiles[c][0.75]:
        return 3
    else:
        return 4

In [12]:
# Применяем функции к данным и считаем итоговую оценку

rfm['r'] = rfm.recency.apply(r_score)
rfm['f'] = rfm.frequency.apply(lambda x: fm_score(x, 'frequency'))
rfm['m'] = rfm.monetary.apply(lambda x: fm_score(x, 'monetary'))

rfm['rfm_score'] = rfm.r.astype(str) + rfm.f.astype(str) + rfm.m.astype(str)
rfm.head()

,CustomerCode,recency,frequency,monetary,r,f,m,rfm_score
0,02213019,19,1,1609.20,1,1,2,112
1,02213042,22,3,9685.48,1,3,4,134
2,02213071,29,1,415.00,1,1,1,111
3,02213088,23,1,305.00,1,1,1,111
4,02213092,25,1,1412.88,1,1,2,112


In [13]:
rfm.query('rfm_score == "111"').shape[0]

10624

### Сколько пользователей попало в сегмент 311?

In [14]:
rfm.query('rfm_score == "311"').shape[0]

4982

### В каком RFM-сегменте самое большое кол-во пользователей?

In [15]:
rfm.rfm_score.value_counts().idxmax()

'111'

### В каком RFM-сегменте самое маленькое кол-во пользователей?

In [16]:
rfm.rfm_score.value_counts().idxmin()

'141'

### Какое количество пользователей попало в самый малочисленный сегмент?

In [17]:
rfm.query('rfm_score == "141"').shape[0]

2